In [537]:
from sklearn import preprocessing as pre
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [538]:
np.random.seed(42)

In [539]:
fish_train_dataset = pd.read_csv("train_fish.csv")
fish_train_dataset.head()

,Weight,Length1,Length2,Length3,Height,Width,Species
0,145.0,20.5,22.0,24.3,6.6339,3.5478,Roach
1,8.7,10.8,11.3,12.6,1.9782,1.2852,Smelt
2,1000.0,41.1,44.0,46.6,12.4888,7.5958,Perch
3,12.2,12.1,13.0,13.8,2.2770,1.2558,Smelt
4,6.7,9.3,9.8,10.8,1.7388,1.0476,Smelt


In [540]:
fish_test_dataset = pd.read_csv("test_fish.csv")
fish_test_dataset.head()

,Weight,Length1,Length2,Length3,Height,Width,Species
0,110.0,20.0,22.0,23.5,5.5225,3.9950,Perch
1,650.0,36.5,39.0,41.4,11.1366,6.0030,Perch
2,567.0,43.2,46.0,48.7,7.7920,4.8700,Pike
3,161.0,22.0,23.4,26.7,6.9153,3.6312,Roach
4,700.0,30.4,33.0,38.3,14.8604,5.2854,Bream


In [541]:
classes = ['Bream', 'Roach', 'Whitefish', 'Parkki', 'Perch', 'Pike', 'Smelt']

In [542]:
def encoding_label(dataset):
    species = []
    
    for fish_type in dataset["Species"]:
        species.append(classes.index(fish_type))
    
    return pd.Series(species)

In [543]:
fish_train_dataset["Species"] = encoding_label(fish_train_dataset)
fish_test_dataset["Species"] = encoding_label(fish_test_dataset)

In [544]:
from sklearn.preprocessing import StandardScaler
 
X_train = fish_train_dataset.drop(["Species", "Width", "Length1", "Weight"], axis=1)
y_train = fish_train_dataset["Species"]
X_test = fish_test_dataset.drop(["Species", "Width", "Length1", "Weight"], axis=1)
y_test = fish_test_dataset["Species"]

In [545]:
scaler = StandardScaler()
transform = scaler.fit(X_train)
X_train = transform.transform(X_train)
X_test = transform.transform(X_test)

In [546]:
def softmax(Z):
    """
    Compute softmax values for each sets of scores in Z.
    each column of Z is a set of score.    
    """
    e_Z = np.exp(Z - np.max(Z, axis = 0, keepdims = True))
    A = e_Z / e_Z.sum(axis = 0)
    return A

In [547]:
def reshape_data(dataList):
    re = np.zeros((dataList.shape[1], dataList.shape[0]))
    
    for idx, data in enumerate(dataList):
        re[:, idx] = data
    
    return re.astype(np.float32)

In [548]:
# One hot coding 
def convert_to_one_hot(y, C):
    re = np.zeros((C, len(y)))
    
    for idx, label in enumerate(y):
        re[:, idx][label] = 1
    
    return re.astype(np.uint8)

In [549]:
# cost or loss function  
def cost(X, Y, W):
    A = softmax(W.T.dot(X))
    return -np.sum(Y*np.log(A))

def grad(X, Y, W):
    A = softmax((W.T.dot(X)))
    E = A - Y
    return X.dot(E.T)
    
def numerical_grad(X, Y, W, cost):
    eps = 1e-6
    g = np.zeros_like(W)
    for i in range(W.shape[0]):
        for j in range(W.shape[1]):
            W_p = W.copy()
            W_n = W.copy()
            W_p[i, j] += eps 
            W_n[i, j] -= eps
            g[i,j] = (cost(X, Y, W_p) - cost(X, Y, W_n))/(2*eps)
    return g 


In [550]:
def softmax_regression(X, y, W_init, eta, tol = 1e-4, max_count = 80000):
    W = [W_init]    
    C = W_init.shape[1]
    Y = convert_to_one_hot(y, C)
    it = 0
    N = X.shape[1]
    d = X.shape[0]
    
    count = 0
    check_w_after = 10
    while count < max_count:
        # mix data 
        mix_id = np.random.permutation(N)
        for i in mix_id:
            xi = X[:, i].reshape(d, 1)
            yi = Y[:, i].reshape(C, 1)
            ai = softmax(np.dot(W[-1].T, xi))
            W_new = W[-1] + eta*xi.dot((yi - ai).T)
            count += 1
            # stopping criteria
            if count%check_w_after == 0:                
                if np.linalg.norm(W_new - W[-check_w_after]) < tol:
                    return W
            W.append(W_new)
    return W

In [551]:
def pred(W, X):
    A = softmax(W.T.dot(X))
    return np.argmax(A, axis = 0)

In [552]:
C = 7

In [553]:
X_train = reshape_data(X_train)

In [554]:
eta = 0.5
d = X_train.shape[0]
W_init = np.random.randn(d, C)
W = softmax_regression(X_train, y_train, W_init, eta)
print(W[-1])

[[ -3.51798782  -4.044408    -0.81242904  -0.05697828   3.61630607
    0.86137743   7.59421037]
 [ -8.8687631    0.12363189   0.4615667  -17.38764675   9.54814999
   12.98088591   1.38223033]
 [ 20.80496544  -0.45322121   5.32434132  18.59102781 -18.88414949
  -16.10458231 -13.11884972]]


In [555]:
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

In [556]:
X = reshape_data(X_test)

In [557]:
test_pred = pred(W[-1], X)

In [558]:
print(classification_report(y_test, test_pred))

              precision    recall  f1-score   support

           0       0.91      1.00      0.95        10
           1       0.75      0.38      0.50         8
           2       0.00      0.00      0.00         2
           3       0.67      1.00      0.80         2
           4       0.53      0.53      0.53        15
           5       0.47      1.00      0.64         7
           6       0.00      0.00      0.00         4

    accuracy                           0.62        48
   macro avg       0.48      0.56      0.49        48
weighted avg       0.58      0.62      0.57        48



c:\Users\Admin\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Admin\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Admin\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo